In [8]:
import cv2
import numpy as np

In [9]:
# Real-world coordinates of your rectangle corners
# Adjust units (cm, m, etc.)
pts_world = np.array([
    [0, 0],       # top-left
    [200, 0],     # top-right
    [200, 100],   # bottom-right
    [0, 100]      # bottom-left
], dtype=np.float32)

clicked_points = []
H = None
calibrated = False

In [10]:
def image_to_world(H, point):
    """Map image pixel (u,v) to real-world (x,y)."""
    uv1 = np.array([point[0], point[1], 1.0])
    xy1 = H @ uv1
    xy1 /= xy1[2]
    return float(xy1[0]), float(xy1[1])

In [11]:
def mouse_callback(event, x, y, flags, param):
    global clicked_points, H, calibrated
    if event == cv2.EVENT_LBUTTONDOWN:
        if not calibrated:
            # Collect the 4 calibration points
            if len(clicked_points) < 4:
                clicked_points.append([x, y])
                print(f"Calibration point {len(clicked_points)}: ({x}, {y})")
        else:
            # After calibration, map any clicked point
            real_coords = image_to_world(H, (x, y))
            print(f"Pixel ({x},{y}) → Real world {real_coords}")
            cv2.circle(param, (x, y), 5, (255, 0, 0), -1)
            cv2.putText(param, f"{real_coords[0]:.1f},{real_coords[1]:.1f}",
                        (x+10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        (255, 0, 0), 1, cv2.LINE_AA)

In [12]:
# Load a snapshot (replace with camera frame if needed)
frame = cv2.imread("snapshot.jpg")
clone = frame.copy()

cv2.namedWindow("Calibration")
cv2.setMouseCallback("Calibration", mouse_callback, clone)

In [13]:
while True:
    display = clone.copy()

    # Draw calibration points
    for i, pt in enumerate(clicked_points):
        cv2.circle(display, tuple(pt), 5, (0, 0, 255), -1)
        if i < len(pts_world):
            rw = pts_world[i]
            cv2.putText(display, f"{rw[0]},{rw[1]}",
                        (pt[0]+10, pt[1]), cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (0, 255, 0), 1, cv2.LINE_AA)

    cv2.imshow("Calibration", display)
    key = cv2.waitKey(1) & 0xFF

    # Press Enter to compute homography once 4 points are set
    if key == 13 and len(clicked_points) == 4 and not calibrated:
        pts_image = np.array(clicked_points, dtype=np.float32)
        H, mask = cv2.findHomography(pts_image, pts_world)
        print("Homography matrix:\n", H)
        calibrated = True
        print("Calibration complete! Now click anywhere to get real-world coords.")

    # ESC to quit
    elif key == 27:
        break

cv2.destroyAllWindows()

Calibration point 1: (869, 42)
Calibration point 2: (1348, 30)
Calibration point 3: (2134, 923)
Calibration point 4: (975, 1166)
Homography matrix:
 [[ 3.70747315e-01 -3.49637147e-02 -3.20710940e+02]
 [ 6.01390717e-03  2.40055128e-01 -1.53084007e+01]
 [-1.16273484e-04  1.55914325e-03  1.00000000e+00]]
Calibration complete! Now click anywhere to get real-world coords.
Pixel (1017,1091) → Real world (7.044231407471597, 97.84354921018969)
Pixel (1001,1158) → Real world (3.6886492978602483, 99.92025066703724)
Pixel (1248,350) → Real world (92.6354831631349, 54.41721353041268)
Pixel (1097,36) → Real world (91.2580828092765, -0.0744794687062951)
Pixel (1134,36) → Real world (106.52437624002158, 0.1659188319599424)
Pixel (1789,789) → Real world (155.75971930531549, 91.41453686237097)
Pixel (1789,789) → Real world (155.75971930531549, 91.41453686237097)
Pixel (1789,789) → Real world (155.75971930531549, 91.41453686237097)


KeyboardInterrupt: 

In [7]:
if len(clicked_points) == 4:
    pts_image = np.array(clicked_points, dtype=np.float32)
    H, mask = cv2.findHomography(pts_image, pts_world)
    print("Homography matrix:\n", H)
else:
    print("Not enough points selected.")

Homography matrix:
 [[ 4.30616088e-01 -3.47894506e-02 -3.79177953e+02]
 [ 7.12865505e-03  2.80987820e-01 -3.92200779e+01]
 [-1.26426938e-04  1.83632697e-03  1.00000000e+00]]
